# Transformaciones Adstock y Diminishing Returns (Contexto)

### Este es un python notebook creado con el fin de facilitar la obtención de las transformaciones adstock y diminishing returns para alimentar el modelo de ecuaciones estructurales

El modelo será alimentado por dos tipos de variables, variables relacionadas a la actividad en medios que llamaremos $X_i$ y variables de control relacionadas principalmente a cuestiones macroeconómicas o fuera de actividad publicitaria y las llamaremos $Z_i,$ y finalmente la variable a explicar $Y$ por lo general ventas.

Para entender las transformaciones adstock debemos pensar en una cuestión, el efecto de la actividad en medios en un día en específico no afectará un único día si no afectara varios días de manera menor cada vez (el efecto olvido) y otra cuestión que debemos entender es el adstock con lag esto corresponde a que invertir una cantidad al tiempo $t$ en el medio $i$ $X_i^t$ no nos genera siempre de manera directa un efecto sobre las ventas en ese mismo dia ($Y^t$), ya que el consumidor no suele hacer la compra inmediatamente, esto es más notorio cuando se trata de carros. En este notebook aplicamos en general Adstock con lag. 

La transformacion de diminishing returns tiene la función de representar de mejor manera la realidad, representando un efecto de saturación, no siempre más inversión significa más ganancia.

Por lo que de $X_i$ obtendremos $X_i'$ la cual corresponde a la variable transformada adstock con lag y finalmente para entrar al modelo añadiremos con la función de diminishing returns $f_i$, finalmente, lo que entra al modelo para las variables de medios es $f_i(X_i')$

# Carga de datos

In [119]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.graph_objs import *
from plotly.subplots import make_subplots
import statsmodels.api as sm

In [120]:
#Para eficientar se puede cargar un archivo excel con las variables suavizadas lista para poder ser transformadas

data = pd.read_excel("Datos/suavizamiento_4formas.xlsx", sheet_name="K-neighbours_smoothing")
data = data.fillna(0)

In [121]:
data.head()

,Date,Ventas_totales_en_piezas,Sell_Out,Ventas_Tradicional_en_piezas,Ventas_Autoservicio_en_piezas,Ventas_Minisúper_en_piezas,"Precio_Tradicional_56g,",Autoservicio_y_Conveniencia_170g_precio,Total_Awareness_conocimiento,Alguna_vez_consumo,...,TRPs_%_Target_Televisa,GRPs_%_Televisa,GRPs_#_Televisa,Inserciones_Televisa,Inversión_MXN_Tv Azteca,TRPs_#_Target_Tv_Azteca,TRPs_%_Target_Tv_Azteca,GRPs_%_Tv_Azteca,GRPs_#_Tv_Azteca,Inserciones_Tv Azteca
0,2019-01-01,26360909.67,27243533.33,20275266.67,1775466.67,4792400.00,12.62,40.10,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-02-01,26360909.67,27243533.33,20275266.67,1775466.67,4792400.00,12.62,40.10,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2019-03-01,25862294.00,25689845.33,19110479.33,1679420.67,4487688.33,12.62,40.96,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2019-04-01,26433231.33,25564302.67,18555510.67,1702655.00,4860946.67,12.69,40.97,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2019-05-01,26030280.33,25466319.33,17931400.00,1752208.00,5320092.00,12.48,40.71,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [122]:
#Variable a utilizar
nombre_variable = "Facebook_Inversión_MXN"
nombre_ventas = "Sell_Out"
try:
    variable = np.array(data[nombre_variable])
    ventas = np.array(data[nombre_ventas])
except:
    print("El nombre de la variable está mal, es sensible a mayúsculas y minisculas")

# Adstock con lag
## Se inplementará con la formula siguiente: 
$$A_t = \frac{\displaystyle \sum_{l=0}^{L-1}w_lx_{t-l}}{\sum_{l=0}^{L-1}w_l}.$$
Donde: 
- $w_l = \lambda^{(l-\theta)^2}.$ 
- $L$: Número de periodos en el futuro en el que dicha inversión tendrá efecto. 
- $\lambda:$ Efecto de decaímiento, es decir, que tanto influye en los siguientes valores. Debe estar entre 0 y 1 
- $\theta:$ Es el lag, cuando tiempo después tendrá el máximo efecto nuestra acción en medios.

In [123]:
############# Rellenar valores ################################################################################################
lambd = 0.9
L = 5
theta = 0
##############################################################################################################################

In [124]:
def get_adstock(X, lambd, L, theta):
    w = np.array([lambd ** ((l - theta) ** 2) for l in range(0,L)])
    A = np.zeros(len(X))
    sum_w = np.sum(w)
    for t in range(len(X)):
        if t < L:
            for l in range(t + 1):
                A[t] += w[l] * X[t - l]
            A[t] =  A[t] / np.sum(w[:(t+1)])
        else:
            for l in range(L):
                A[t] +=  w[l] * X[t - l]
            A[t] /= sum_w
    return A

In [125]:
Adstock = get_adstock(variable, lambd = lambd, L = L, theta = theta)

print("\tMedia Variable original: " + str(float(np.mean(variable))) + " Vs. " + "Media Adstock: "+ str(float(np.mean(Adstock))))

fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(len(variable))), y=variable, name=nombre_variable))
fig.add_trace(go.Scatter(x=list(range(len(Adstock))), y= Adstock, name = ("Adstock " + nombre_variable)))
fig.update_layout(title="Transformación")
fig.update_scenes(xaxis_visible=True, yaxis_visible=False)

	Media Variable original: 153832.87048780487 Vs. Media Adstock: 154353.1214825062


### Obtención de $\theta$
El de mayor correlación parece ser el indicado, ojo debe tener sentido, puede ser coincidencia

In [126]:
def lag(X, l, rellenar=True):
    if l == 0:
        return X
        pass
    X_lag = np.zeros(len(X))
    X_lag[l:] = X[:-l]
    if(rellenar):
        X_lag[:l] = np.mean(X)
    return X_lag
    

In [127]:
################### LAG PARA QUE SE VEA EN GRÁFICA #######################

lag_graf = 7
rellenar= True   #Rellenar vacios con media

###########################################################################

In [128]:
correlaciones = np.zeros(11)
for i in range(0,11):
    correlaciones[i] = np.corrcoef(lag(variable, i, rellenar), ventas)[0][1]


fig = make_subplots(rows=1, cols=2, specs=[[{"secondary_y": True}, {"secondary_y": True}]])

fig.add_trace(go.Scatter(
    x=list(range(len(ventas))),
    y=ventas,name=nombre_ventas
    ), row=1, col= 1, secondary_y=False,)

fig.add_trace(go.Scatter(
    x=list(range(len(variable))),
    y=lag(variable,lag_graf, rellenar),
    name=nombre_variable + " lag:" + str(lag_graf) ),  row=1, col= 1, secondary_y=True,)

fig.add_trace(go.Bar(x=list(range(0,11)), y=correlaciones, name='Correlacion'), row=1,col=2)
fig.show()    

### Cálculo de $\lambda,$ en este debemos tener cuidado.

##### Aca se gráfica del astock con varlos $\theta$ y $\lambda$ contra las ventas

In [117]:
range_lambd = np.arange(0.1,1, step=0.05)
Adst_lambd = np.zeros((len(range_lambd), len(variable)))
corr_adst = np.zeros(len(range_lambd))
for i in range(len(range_lambd)):
    Adst_lambd[i,:] = get_adstock(variable, range_lambd[i], L=L, theta=theta)
    corr_adst[i] = np.corrcoef(Adst_lambd[i,:], ventas)[0][1]

fig = go.Figure()
fig.add_trace(go.Scatter(x=range_lambd, y= corr_adst, name = ("Adstock " + nombre_variable)))
fig.update_layout(title="Correlación con Adstock para decay \lambda")
fig.update_scenes(xaxis_visible=True, yaxis_visible=False)

#### Aca se gráfica el R-squared de regressión del adstock de $\theta$ y $\lambda$ con las ventas

In [118]:
R = np.zeros(len(range_lambd))
for i in range(len(range_lambd)):
    explain = sm.add_constant(Adst_lambd[i,:],prepend=False)
    mod = sm.OLS(ventas, explain)
    res = mod.fit()
    R[i] = res.rsquared
    
fig = go.Figure()
fig.add_trace(go.Scatter(x=range_lambd, y= R, name = ("Adstock " + nombre_variable)))
fig.update_layout(title="R-squared con Adstock para decay \lambda")
fig.update_scenes(xaxis_visible=True, yaxis_visible=False)

# Shape effect. Curve of diminishing returns
## Se le aplicán a las variables con el efecto adstock ya incluido
### En esta parte se implementará el efecto d saturación de inversión en medios, no siempre más inversión es más dinero en la misma proporción, esto es importante para el ROI.
### Para esto tenemos varías opciones, aca se presentan.

<style>
    th, td { 
      font-size: 50px;
      border-collapse: collapse;
      border-width:3px
    }
    </style>

| Nombre Función |Parámetros        |  Función  |
|:---------------|:----------------:|:---------:|
|Logaritmica     | Ninguno          |$\log(x)$|
|Exp. Negativa   |$\alpha$: Tasa de saturación| $1-$ $e^{-\alpha x}$  |
|Logistica       |$\alpha$: Tasa de saturación <br> $\gamma$: Punto de inflexión de la curva | $$\frac{1}{1 + e^{-\alpha(x-\gamma)}}$$|

### Un valor para iniciar puede ser para $\alpha \sim$ $\frac{1}{\text{Media de Adstock o Max}}$
### EN la función se colocan como el nombre que están aquí entre comillas

In [272]:
############################################# RELLENAR AQUÍ ###########################################
tipo = "Logistica"
alpha = 0.00009
gamma = 100000
#######################################################################################################

In [273]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

def get_shape_effect(variable_adstock, tipo="Exp. Negativa", alpha=0.5, gamma=0):
    if tipo == "Logaritmica":
        return np.log(variable_adstock)
    if tipo == "Exp. Negativa":
        return (1- np.exp(-alpha * variable_adstock))
    if tipo == "Logistica":
        return (1 / (1 + np.exp(-alpha * (variable_adstock - gamma))))
    
    print("ERROR !!!!!!!!!! Escribiste mal el nombre de la función a aplicar, por favor corrige.")
    return variable_adstock

fig = px.scatter(x=Adstock, y=get_shape_effect(Adstock, tipo = tipo, alpha = alpha, gamma = gamma), title="Shappe effect vs Adstock",)
fig.update_traces(mode='markers', marker_line_width=2, marker_size=10)
fig.update_yaxes(
    title="Transformación 2"
)
fig.update_xaxes(title = nombre_variable + " Adstock ")
print(f"{bcolors.OKCYAN}" "Correlación Adstock con ventas: \t" f"{bcolors.ENDC}"+ str(np.corrcoef(Adstock,ventas)[0][1]))
print(f"{bcolors.OKCYAN}" "Correlación Shape effect con ventas:\t" f"{bcolors.ENDC}" + str(np.corrcoef(get_shape_effect(Adstock, tipo = tipo, alpha = alpha, gamma = gamma), ventas)[0][1]))
fig.show()

Correlación Adstock con ventas: 	0.4539625499371439
Correlación Shape effect con ventas:	0.485061228747101
